* Calculate mouth Kelvin number.

In [1]:
import numpy as np
import netCDF4 as nc
from salishsea_tools import (nc_tools,viz_tools,tidetools,geo_tools)
import matplotlib.pyplot as plt
import FroudeNumber as FN
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set(style="whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
%matplotlib inline

In [2]:
grid6 = nc.Dataset('/ocean/jieliu/research/meopar/river-treatment/bathy_meter_SalishSea6.nc')
X = grid6.variables['nav_lon'][:, :]
Y = grid6.variables['nav_lat'][:, :]
bathy = grid6.variables['Bathymetry'][:, :]
## tmask
mesh = nc.Dataset('/data/jieliu/MEOPAR/river-treatment/oct8_101e061e05/mesh_mask.nc')
tmask = mesh.variables['tmask'][0,:,380:510,240:397]
np_mask = np.abs(1-tmask) 
e3t = mesh.variables['e3t'][0,:,380:510,240:397]

In [3]:
## May results
all_T = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/May2015combineall/may2015all_T.nc','r')
all_U = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/May2015combineall/may2015all_U.nc','r')

In [4]:
## Oct results
octall_T = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/Oct2014combineall/oct2014all_T.nc','r')
octall_U = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/Oct2014combineall/oct2014all_T.nc','r')

In [5]:
## Jan results
janall_T = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/Jan2016combineall/jan2016all_T.nc','r')
janall_U = nc.Dataset('/data/jieliu/MEOPAR/SurfaceCurrent/Jan2016combineall/jan2016all_U.nc','r')

# Functions

In [6]:
def calculate_density(t, s):
    """Caluclates the density given temperature in deg C (t)
    and salinity in psu (s).

    returns the density as an array (rho)
    """
    rho = (
        999.842594 + 6.793952e-2 * t
        - 9.095290e-3 * t*t + 1.001685e-4 * t*t*t
        - 1.120083e-6 * t*t*t*t + 6.536332e-9 * t*t*t*t*t
        + 8.24493e-1 * s - 4.0899e-3 * t*s
        + 7.6438e-5 * t*t*s - 8.2467e-7 * t*t*t*s
        + 5.3875e-9 * t*t*t*t*s - 5.72466e-3 * s**1.5
        + 1.0227e-4 * t*s**1.5 - 1.6546e-6 * t*t*s**1.5
        + 4.8314e-4 * s*s)
    return rho

In [7]:
def Calculate_ReducedGravity(rho,dep,j,i):
    """
    calculate reduced gravity, g' = g(rho0 - integral(rho dz)/integral(dz))/rho0
    
    rho: density calculated based on T and S
    
    dep: level of depth selected
    
    j: along-strait index 
    
    i: across-strait index
    
    return: g_prime
    """
    g = 9.80665
    rho0 = 1023.8
    rho_dz = np.zeros([SSH.shape[0],dep])
    dz = np.zeros(dep)
    for t in range(SSH.shape[0]):
        for z in range(dep):
            adj_ssh = 1+SSH[t,j,i]/np.sum(e3t[:,j,i]*tmask[:,j,i],axis = 0)
            rho_dz[t,z] = rho[t,z] * e3t[z, j,i]* adj_ssh * tmask[z,j,i]
            dz[z]= e3t[z, j,i]* adj_ssh * tmask[z,j,i]
    integral_rho_dz = np.nanmean(np.nansum(rho_dz, axis = 1))
    integral_dz = np.nansum(dz)
    g_prime = np.sqrt(g*(rho0-integral_rho_dz/integral_dz)/rho0)
    return g_prime

In [8]:
def Calculate_RossbyDeformationRadius(g_prime, h):
    """
    """
    f = 1e-4
    RDR = np.sqrt(g_prime*h)/f
    return RDR

# calculation

In [9]:
## In May
T = all_T.variables['votemper'][:,:,40,72] ## select mouth grid point
S = all_T.variables['vosaline'][:,:,40,72]
DEPTH = all_T.variables['deptht']
SSH = all_T.variables['sossheig']
rho = calculate_density(T,S)
g_prime_may = Calculate_ReducedGravity(rho,8,40,72)
RDR_may = Calculate_RossbyDeformationRadius(g_prime_may,8)
print('Rossby deformation radius is ', RDR_may/1e3, ' km in May')

Rossby deformation radius is  26.680952049  km in May


In [10]:
## In oct
T = octall_T.variables['votemper'][:,:,40,72]
S = octall_T.variables['vosaline'][:,:,40,72]
SSH = octall_T.variables['sossheig']
rho = calculate_density(T,S)
g_prime_oct = Calculate_ReducedGravity(rho,8,40,72)
RDR_oct = Calculate_RossbyDeformationRadius(g_prime_oct,8)
print('Rossby deformation radius is ', RDR_oct/1e3, ' km in Oct')

Rossby deformation radius is  21.6319702359  km in Oct


In [11]:
# In Jan
T = janall_T.variables['votemper'][:,:,40,72]
S = janall_T.variables['vosaline'][:,:,40,72]
SSH = janall_T.variables['sossheig']
rho = calculate_density(T,S)
g_prime_jan = Calculate_ReducedGravity(rho,8,40,72)
RDR_jan = Calculate_RossbyDeformationRadius(g_prime_jan,8)
print('Rossby deformation radius is ', RDR_jan/1e3, ' km in Jan')

Rossby deformation radius is  nan  km in Jan


# Test why in Jan Rossby deformation radius not work??

In [12]:
dep = 8
g = 9.80665
rho0 = 1023.8
j = 40 ; i = 72
rho_dz = np.zeros([SSH.shape[0],dep])
dz = np.zeros(dep)
for t in range(SSH.shape[0]):
    for z in range(dep):
        adj_ssh = 1+SSH[t,j,i]/np.sum(e3t[:,j,i]*tmask[:,j,i],axis = 0)
        rho_dz[t,z] = rho[t,z] * e3t[z, j,i]* adj_ssh * tmask[z,j,i]
        dz[z]= e3t[z, j,i]* adj_ssh * tmask[z,j,i]
integral_rho_dz = np.nanmean(np.nansum(rho_dz, axis = 1))
integral_dz = np.nansum(dz)
g_prime = np.sqrt(g*(rho0-integral_rho_dz/integral_dz)/rho0)
print('rho0 is', rho0)
print('integral of rho over depths divided by integral of depths is',integral_rho_dz/integral_dz )

rho0 is 1023.8
integral of rho over depths divided by integral of depths is 1104.657372
